In [3]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import random

#from ggplot import *
from scipy.stats import logistic
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import *
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
#import graphviz
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR

# 한글 폰트 실행
from matplotlib import rcParams
from matplotlib import font_manager,rc
rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)


In [4]:
DATA_PATH = './dataset/CARD/'
DATA_PATH_2 = './dataset/CUSTOMER/'
card_21 = pd.read_csv(DATA_PATH + 'card_21.csv', encoding="euc-kr")
#card_22 = pd.read_csv(DATA_PATH + 'card_22.csv', encoding="euc-kr")
#card_23 = pd.read_csv(DATA_PATH + 'card_23.csv', encoding="euc-kr")
customer_21 = pd.read_csv(DATA_PATH_2 + 'customer_21.csv', encoding="euc-kr")
#customer_22 = pd.read_csv(DATA_PATH_2 + 'customer_22.csv', encoding="euc-kr")
#customer_23 = pd.read_csv(DATA_PATH_2 + 'customer_23.csv', encoding="euc-kr")

In [5]:
card_21.head(3)

,거래년월,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액
0,202112,4080934272,일반한식,경상북도,경주시,5번이하,20000
1,202112,4116127380,일반한식,경상북도,경주시,5번이하,20000
2,202112,2914424833,일반한식,경상북도,경주시,5번이하,20000


In [6]:
card_21['가맹점업종명'].value_counts()

가맹점업종명
일반한식        8751749
인터넷P/G      5622089
슈퍼마켓        5174522
편 의 점       5103418
서양음식        4763634
             ...   
전자상거래상품권       1034
상품권전문판매         369
맞춤복점            347
부동산중개·임대        344
냉열기기            227
Name: count, Length: 205, dtype: int64

for col in card_21.select_dtypes(include=['int64', 'float64']).columns:
    if 'int' in str(card_21[col].dtype):
        card_21[col] = card_21[col].astype('int32')
    elif 'float' in str(card_21[col].dtype):
        card_21[col] = card_21[col].astype('float32')

for col in customer_21.select_dtypes(include=['int64', 'float64']).columns:
    if 'int' in str(customer_21[col].dtype):
        customer_21[col] = customer_21[col].astype('int32')
    elif 'float' in str(customer_21[col].dtype):
        customer_21[col] = customer_21[col].astype('float32')



In [30]:
card_test_21 = card_21.drop(columns=['가맹점_시군구', '거래년월', '가맹점_광역시도','승인건수'])

In [31]:
card_test_21

,고객ID,가맹점업종명,승인금액
0,4080934272,일반한식,20000
1,4116127380,일반한식,20000
2,2914424833,일반한식,20000
3,2950007638,일반한식,60000
4,2931907302,일반한식,10000
...,...,...,...
60672637,9436485459,일식·회집,40000
60672638,4079854553,일식·회집,90000
60672639,9986820487,일식·회집,10000
60672640,8230823000,일식·회집,20000


In [28]:
customer_21.head(3)

,기준년월,고객ID,연령대,성별,고객등급,자택_시도,자택_시군구,수신_요구불예금,수신_거치식예금,수신_적립식예금,수신_펀드,수신_외화예금,대출금액
0,202112,9999998161,60대이상,여,일반등급,대구광역시,달서구,1400000,0,0,0,0,0
1,202112,9999997589,60대이상,남,일반등급,경상북도,경주시,0,10000000,500000,0,0,0
2,202112,9999991579,60대이상,여,일반등급,경상북도,성주군,0,30000000,0,0,0,0


In [29]:
customer_test_21 = customer_21.drop(columns=['기준년월', '성별','고객등급', '자택_시도', '자택_시군구', '수신_요구불예금','수신_거치식예금','수신_적립식예금','수신_펀드', '수신_외화예금','대출금액'])
customer_test_21.head(3)

,고객ID,연령대
0,9999998161,60대이상
1,9999997589,60대이상
2,9999991579,60대이상


In [34]:
customer_test_21['연령대'].value_counts()

연령대
60대이상    5286593
50대      4080736
40대      3822171
20대      3099174
30대      3013611
10대미만    1533707
Name: count, dtype: int64

In [33]:
#card_customer_21= pd.merge(customer_test_21, card_test_21, on='고객ID', how='inner')
#card_customer_21

In [35]:
customer_test_21_5060 = customer_test_21[customer_test_21['연령대'].isin(['50대', '60대이상'])]
customer_test_21_5060.head(5)

,고객ID,연령대
0,9999998161,60대이상
1,9999997589,60대이상
2,9999991579,60대이상
3,9999989448,60대이상
4,9999988325,60대이상


In [36]:
card_customer_21_5060= pd.merge(customer_test_21_5060, card_test_21, on='고객ID', how='inner')
card_customer_21_5060

MemoryError: Unable to allocate 2.05 GiB for an array with shape (275163705,) and data type int64

In [37]:
customer_test_21_60 = customer_test_21[customer_test_21['연령대'].isin(['60대이상'])]
customer_test_21_60.head(3)

,고객ID,연령대
0,9999998161,60대이상
1,9999997589,60대이상
2,9999991579,60대이상


In [38]:
card_customer_21_60= pd.merge(customer_test_21_60, card_test_21, on='고객ID', how='inner')
card_customer_21_60

MemoryError: Unable to allocate 854. MiB for an array with shape (1, 111995538) and data type int64

In [39]:
card_test_21.head(3)

,고객ID,가맹점업종명,승인금액
0,4080934272,일반한식,20000
1,4116127380,일반한식,20000
2,2914424833,일반한식,20000


In [41]:
card_test_21_id = card_21.drop(columns=['가맹점_시군구', '거래년월', '가맹점_광역시도','승인건수','가맹점업종명','승인금액'])
card_test_21_id.head(3)

,고객ID
0,4080934272
1,4116127380
2,2914424833


In [42]:
card_customer_21_60_id= pd.merge(customer_test_21_60, card_test_21_id, on='고객ID', how='inner')
card_customer_21_60_id.head(3)

MemoryError: Unable to allocate 854. MiB for an array with shape (111995538,) and data type int64

In [43]:
import time

def execution_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        hours, rem = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"Execution time for {func.__name__}: {int(hours)}h {int(minutes)}m {seconds:.2f}s")
        return result
    return wrapper

customer_test_21 = customer_21.drop(columns=['기준년월', '성별','고객등급', '자택_시도', '자택_시군구', '수신_요구불예금','수신_거치식예금','수신_적립식예금','수신_펀드', '수신_외화예금','대출금액'])

customer_test_21_60 = customer_test_21[customer_test_21['연령대'].isin(['60대이상'])]

card_test_21_id = card_21.drop(columns=['가맹점_시군구', '거래년월', '가맹점_광역시도','승인건수','가맹점업종명','승인금액'])

card_customer_21_60_id= pd.merge(customer_test_21_60, card_test_21_id, on='고객ID', how='inner')
card_customer_21_60_id.head(3)

In [44]:
customer_21.head(3)

,기준년월,고객ID,연령대,성별,고객등급,자택_시도,자택_시군구,수신_요구불예금,수신_거치식예금,수신_적립식예금,수신_펀드,수신_외화예금,대출금액
0,202112,9999998161,60대이상,여,일반등급,대구광역시,달서구,1400000,0,0,0,0,0
1,202112,9999997589,60대이상,남,일반등급,경상북도,경주시,0,10000000,500000,0,0,0
2,202112,9999991579,60대이상,여,일반등급,경상북도,성주군,0,30000000,0,0,0,0


In [45]:
card_21.head(3)

,거래년월,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액
0,202112,4080934272,일반한식,경상북도,경주시,5번이하,20000
1,202112,4116127380,일반한식,경상북도,경주시,5번이하,20000
2,202112,2914424833,일반한식,경상북도,경주시,5번이하,20000


In [47]:
customer_test_21.head(3)

,고객ID,연령대
0,9999998161,60대이상
1,9999997589,60대이상
2,9999991579,60대이상


In [48]:
card_test_21.head(3)

,고객ID,가맹점업종명,승인금액
0,4080934272,일반한식,20000
1,4116127380,일반한식,20000
2,2914424833,일반한식,20000


In [49]:
card_21.head(3)

,거래년월,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액
0,202112,4080934272,일반한식,경상북도,경주시,5번이하,20000
1,202112,4116127380,일반한식,경상북도,경주시,5번이하,20000
2,202112,2914424833,일반한식,경상북도,경주시,5번이하,20000


# 다시

In [53]:
customer_21.head(3)

,기준년월,고객ID,연령대,성별,고객등급,자택_시도,자택_시군구,수신_요구불예금,수신_거치식예금,수신_적립식예금,수신_펀드,수신_외화예금,대출금액
0,202112,9999998161,60대이상,여,일반등급,대구광역시,달서구,1400000,0,0,0,0,0
1,202112,9999997589,60대이상,남,일반등급,경상북도,경주시,0,10000000,500000,0,0,0
2,202112,9999991579,60대이상,여,일반등급,경상북도,성주군,0,30000000,0,0,0,0


In [54]:
customer_21['연령대'].value_counts()

연령대
60대이상    5286593
50대      4080736
40대      3822171
20대      3099174
30대      3013611
10대미만    1533707
Name: count, dtype: int64

In [ ]:
# 고객 - 고객id, 연령대만 남기고 삭제
customer_21_test2 = customer_21.loc[:, ['고객ID', '연령대']]
customer_21

In [ ]:
customer_test_21 = customer_21.drop(columns=['기준년월', '성별','고객등급', '자택_시도', '자택_시군구', '수신_요구불예금','수신_거치식예금','수신_적립식예금','수신_펀드', '수신_외화예금','대출금액'])

customer_test_21_60 = customer_test_21[customer_test_21['연령대'].isin(['60대이상'])]

card_test_21_id = card_21.drop(columns=['가맹점_시군구', '거래년월', '가맹점_광역시도','승인건수','가맹점업종명','승인금액'])

card_customer_21_60_id= pd.merge(customer_test_21_60, card_test_21_id, on='고객ID', how='inner')
card_customer_21_60_id.head(3)

In [52]:
# 고객 데이터 - 60대
customer_21_test2_60 = customer_21.loc[:, ['고객ID', '연령대']]
customer_21_test2_60.head(3)

,고객ID,연령대
0,9999998161,60대이상
1,9999997589,60대이상
2,9999991579,60대이상


In [ ]:
# 고객 데이터 - 50대
customer_21_test2_50 = customer_21.loc[:, ['고객ID', '연령대']]
customer_21_test2_60.head(3)

In [50]:
card_21_test2 = card_21.loc[:, ['고객ID', '가맹점업종명','승인금액']]
card_21_test2.head(3)

,고객ID,가맹점업종명,승인금액
0,4080934272,일반한식,20000
1,4116127380,일반한식,20000
2,2914424833,일반한식,20000
